In [5]:
import numpy as np
import pandas as pd
import cvxpy as cp
import mosek
import matplotlib.pyplot as plt
import affine_approx as af
import importlib
import scipy.stats
from scipy.stats import norm, chi2, wishart, beta, lognorm

In [17]:
def kl_ca_conj(x):
    return(1-np.exp(-x))

def rk_eval(X,delta, phi_conj):
    n = len(X)
    som = 0
    X = np.sort(X)
    X_diff = np.concatenate((np.array([X[0]]),np.diff(X)))
    for i in range(n):
        som = som + phi_conj(delta*(n-i)/n)*X_diff[i]
    return(som/delta)

def sd(X):
    n = len(X)
    som = 0
    m = np.mean(X)
    for i in range(n):
        som = som + (X[i]-m)**2
    return(np.sqrt(som/n))

def miniance(X):
    n = len(X)
    som = 0
    X = np.sort(X)
    X_diff = np.concatenate((np.array([X[0]]),np.diff(X)))
    for i in range(n):
        som = som + ((n-i)/n)**2*X_diff[i]
    return(som)

def h_ddro(x,delta):
    teller = 1-np.exp(-delta*x)
    noemer = 1-np.exp(-delta)
    return(teller/noemer)


def argmax_kl_dtconj(x2,x1,delta):
    frac = (np.exp(-delta*x1)-np.exp(-delta*x2))/(x2-x1)
    return(-np.log(frac/delta)/delta)

def max_kl_dtconj(x2,x1,x,delta):
    frac = (h_ddro(x2,delta)-h_ddro(x1,delta))/(x2-x1)
    return(h_ddro(x,delta)-frac*(x-x1)-h_ddro(x1,delta))

In [27]:
def generate_samples(delta, N):
    samples = []
    for _ in range(N):
        random_number = np.random.rand()
        if random_number < delta:
            samples.append(0)  # With probability delta, append 0
        else:
            samples.append(np.random.choice([-1, 1]))  # With probability 1-delta, append -1 or 1 
    return samples


def generate_samples_with_perturbations(N, delta_gen, epsilon_gen, M):
    # generate samples X = X1 + Z
    X_samples = np.zeros(N)
    
    # Generate samples for X1
    # X1 = 0 with probability delta_gen, and with probability (1 - delta_gen), X1 = 1 or -1
    X1_samples = np.where(np.random.rand(N) < delta_gen, 
                          0, 
                          np.random.choice([-1, 1], size=N))
    
    # Generate samples for Z
    # Z = 0 with probability (1 - epsilon_gen), and Z = M with probability epsilon_gen
    Z_samples = np.where(np.random.rand(N) < epsilon_gen, 
                         M, 
                         0)
    
    X_samples = X1_samples + Z_samples
    
    return X_samples

def median_saa(X):
    n = len(X)
    theta = cp.Variable()
    objective = cp.Minimize(cp.sum(cp.abs(theta - X)) / n)
    problem = cp.Problem(objective)
    problem.solve()
    return theta.value, problem.value


def median_dro(X, r):
    n = len(X)
    lambda_var = cp.Variable(nonneg=True)
    theta = cp.Variable()
    eta = cp.Variable()
    t = cp.Variable(n)

    
    objective = cp.Minimize(eta + lambda_var * (r - 1) + cp.sum(t) / n)
    abs_diff = cp.abs(theta - X)
    kl_div_term = -cp.kl_div(t, lambda_var) - lambda_var + t
    constraints = [t >= 0, abs_diff - eta <= kl_div_term]

    problem = cp.Problem(objective, constraints)

    problem.solve()

    return theta.value, problem.value

def median_ddro(R, slope, const):
    n = len(R)
    K = len(slope)
    theta = cp.Variable()
    beta = cp.Variable()
    lbda = cp.Variable((n, K), nonneg=True)
    v = cp.Variable(K, nonneg=True)
    p = np.ones(n) / n
    c = cp.Variable()
    abs_diff = cp.abs(theta - R)
    constraints = [
        abs_diff - cp.sum(lbda, axis=1) - beta <= 0,  
        lbda <= cp.reshape(v, (1, K),order='F')  # Ensure v is broadcasted to shape (1, K)
    ]
    
    constraints.append(beta + v @ const + p @ (lbda @ slope) <= c)
    obj = cp.Minimize(c)
    prob = cp.Problem(obj, constraints)
    prob.solve()
    
    return theta.value, prob.value



In [43]:
# Robust median estimation
M = 300
eps_gen = 0.009  # Set epsilon to your desired value
delta_gen = 0.01  # Set delta to your desired value
N = 1000  # Number of samples to generate
eps = 0.0001
rep = 100
saa_count = 0
dro_count = 0
ddro_count = 0
np.random.seed(250)
for i in range(rep):
    #samples = generate_samples(delta_gen,N)   #### Use this to generate samples without perturbation
    samples = generate_samples_with_perturbations(N, delta_gen, eps_gen, M)      #### Use this to generate samples with perturbation
    r = chi2.ppf(0.999,1)/(2*N)
    r2 = np.sqrt(2*chi2.ppf(0.999,1))
    delta = r2/np.sqrt(N)
    x_points = af.affine_approx(eps,argmax_kl_dtconj,max_kl_dtconj,delta)
    if len(x_points)==2:
        print('true')
    else:
        [slope, const] = af.makepoints(h_ddro,x_points,delta)
    sol_saa = median_saa(samples)[0]
    sol_dro = median_dro(samples,r)[0]
    sol_ddro = median_ddro(samples, slope, const)[0]
    if np.abs(sol_saa) <= 0.01:
        saa_count = saa_count + 1
    if np.abs(sol_dro) <= 0.01:
        dro_count = dro_count + 1
    if np.abs(sol_ddro) <= 0.01:
        ddro_count = ddro_count + 1
    if i% 50 == 0:
        print(i)
    

0
50


In [45]:
print(np.mean(saa_count))
print(np.mean(dro_count))
print(np.mean(ddro_count))

22.0
22.0
84.0
